In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI管道：使用BigQuery源和目的地进行训练和批量预测自定义表格分类模型

<table align="left">

  <td>
<a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/custom_tabular_train_batch_pred_bq_pipeline.ipynb" target='_blank'>
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
<a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/custom_tabular_train_batch_pred_bq_pipeline.ipynb" target='_blank'>
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/pipelines/custom_tabular_train_batch_pred_bq_pipeline.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>  
</table>

*备注：此笔记本使用KFP 1.x和GCPC 1.x。我们建议使用2.x*

## 概述

这本笔记本演示了在Vertex AI管道内为自定义表格分类模型执行训练和批量预测。批量预测作业从BigQuery源获取数据，并将结果写入BigQuery目的地。

了解更多关于[Vertex AI管道](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)和[Vertex AI批量预测组件](https://cloud.google.com/vertex-ai/docs/pipelines/batchprediction-component)。

### 目标

在本教程中，您将通过使用 `google_cloud_pipeline_components` 通过 Vertex AI 管道训练一个 scikit-learn 表格分类模型，并为其创建批量预测作业。批量预测作业的源数据和目标数据存储在 BigQuery 中。

本教程使用以下 Google Cloud ML 服务和资源:

- Vertex AI `Pipelines`
- Vertex AI `Datasets`
- Vertex AI `Training`
- Vertex AI `Model Registry`
- Vertex AI `Batch Predictions`

执行的步骤包括:

- 在 BigQuery 中创建一个数据集。
- 从源数据集中保留一些数据用于批量预测。
- 为训练应用程序创建自定义的 Python 包。
- 将 Python 包上传到 Cloud Storage。
- 创建一个 Vertex AI 管道，其中包括:
    - 从源数据集创建一个 Vertex AI 数据集。
    - 在数据集上训练一个 scikit-learn 随机森林分类模型。
    - 将训练完成的模型上传到 Vertex AI Model Registry。
    - 使用模型在测试数据上运行一个批量预测作业。
- 检查在 BigQuery 的目标表中的预测结果。
- 清理本笔记本中创建的资源。

### 数据集

这本笔记本使用的[人口普查收入数据集](https://archive.ics.uci.edu/ml/datasets/Census+Income)可以在BigQuery位置`bigquery-public-data.ml_datasets.census_adult_income`上公开获取。它包括以下字段：

- `age`：年龄。
- `workclass`：就业性质。
- `functional_weight`：个体在原始人口普查数据中的样本权重。根据其人口特征与整体人口估计相比，他们被包含在这个数据集中的可能性有多大。
- `education`：完成的教育水平。
- `education_num`：根据教育字段的值估算完成的教育年限。
- `marital_status`：婚姻状况。
- `occupation`：职业类别。
- `relationship`：与家庭的关系。
- `race`：种族。
- `sex`：性别。
- `capital_gain`：资本收益金额。
- `capital_loss`：资本损失金额。
- `hours_per_week`：每周工作小时数。
- `native_country`：出生国家。
- `income_bracket`：根据收入为“>50K”或“<=50K”。

### 费用
该教程使用 Google Cloud 的可计费组件：

* Vertex AI
* BigQuery
* Cloud Storage

了解有关[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)，[BigQuery 价格](https://cloud.google.com/bigquery/pricing)，[Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing 计算器](https://cloud.google.com/products/calculator/)，根据您的预期使用情况生成费用估算。

安装以下包以执行此笔记本。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-bigquery \
                                 pandas \
                                 pyarrow \
                                 'kfp<2' \
                                 'google-cloud-pipeline-components<2' 

! pip3 install --quiet db-dtypes 

### 仅限Colab：取消注释以下单元格以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

UUID

如果您正在进行实时教程会话，您可能正在使用共享的测试帐户或项目。为了避免用户在创建的资源之间的名称冲突，您为每个实例会话创建一个 uuid，并将其附加到您在本教程中创建的资源名称上。

In [1]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

## 在开始之前

### 设置您的谷歌云项目

**无论您使用什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建账户时，您会获得$300的免费信用用于计算/存储成本。

2. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API]。

4. 如果您是在本地运行这个笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

#### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 参考支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


### 区域

您也可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [3]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的 Google Cloud 帐户

根据您的 Jupyter 环境，您可能需要手动进行身份验证。请按照下面的相关说明操作。

**1. Vertex AI Workbench**
* 由于您已经验证过了，无需进行额外操作。

**2. 本地 JupyterLab 实例，请取消注释并运行：**

In [4]:
# ! gcloud auth login

3. 协作，取消注释并运行：

In [5]:
# from google.colab import auth
# auth.authenticate_user()

查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples中为您的服务帐号授予云存储权限。

创建一个云存储桶

创建一个存储桶来存储诸如数据集之类的中间产品。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在时才运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

服务账户

您使用服务账户来创建 Vertex AI Pipeline 作业。如果您不想使用项目的 Compute Engine 服务账户，请将 `SERVICE_ACCOUNT` 设置为另一个服务账户的ID。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

为Vertex AI Pipelines设置服务账户访问

运行以下命令，为您的服务账户授予读取和写入管道工件的权限，这些工件存储在您在之前步骤中创建的存储桶中。您只需要针对每个服务账户运行此步骤一次。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 导入库

导入Vertex AI Python SDK和其他必需的Python库。

In [ ]:
import os

from google.cloud import aiplatform, bigquery
from kfp.dsl import pipeline
from kfp.v2 import compiler

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 版本的 Vertex AI SDK。

In [ ]:
# Initialize Vertex AI SDK
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

# Initialize BigQuery client
bq_client = bigquery.Client(
    project=PROJECT_ID,
    credentials=aiplatform.initializer.global_config.credentials,
)

定义常量

在训练模型、创建和运行管道时设置你所需要的常量。

In [ ]:
# Source of the dataset
DATA_SOURCE = "bq://bigquery-public-data.ml_datasets.census_adult_income"
# Set name for the managed Vertex AI dataset
DATASET_DISPLAY_NAME = f"adult_census_dataset_{UUID}"
# BigQuery Dataset name
BQ_DATASET_ID = f"income_prediction_{UUID}"
# Set name for the BigQuery source table for batch prediction
BQ_INPUT_TABLE = f"income_test_data_{UUID}"
# Set the size(%) of the train set
TRAIN_SPLIT = 0.9
# Provide the container for training the model
TRAINING_CONTAINER = "us-docker.pkg.dev/vertex-ai/training/scikit-learn-cpu.0-23:latest"
# Provide the container for serving the model
SERVING_CONTAINER = "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-23:latest"
# Set the display name for training job
TRAINING_JOB_DISPLAY_NAME = f"income_classify_train_job_{UUID}"
# Model display name for Vertex AI Model Registry
MODEL_DISPLAY_NAME = f"income_classify_model_{UUID}"
# Set the name for batch prediction job
BATCH_PREDICTION_JOB_NAME = f"income_classify_batch_pred_{UUID}"
# Dispaly name for the Vertex AI Pipeline
PIPELINE_DISPLAY_NAME = f"income_classfiy_batch_pred_pipeline_{UUID}"
# Filename to compile the pipeline to
PIPELINE_FILE_NAME = f"{PIPELINE_DISPLAY_NAME}.json"

创建一个BigQuery数据集
在本教程中，您的批量预测作业的输入和输出需要存储在BigQuery中。因此，您需要在BigQuery中创建一个数据集来存储它们。

In [ ]:
# Create a BQ dataset
bq_dataset = bigquery.Dataset(f"{PROJECT_ID}.{BQ_DATASET_ID}")
bq_dataset = bq_client.create_dataset(bq_dataset)
print(f"Created dataset {bq_client.project}.{bq_dataset.dataset_id}")

## 为批量预测创建测试数据

查询公共数据集源并在创建的BigQuery数据集中创建一个测试集。

对于批量预测，您的测试集是通过随机选择源数据集的一小部分（1-`TRAIN_SPLIT`）来创建的。

In [ ]:
# Query to create a test set from the source table
query = f"""
CREATE OR REPLACE TABLE
  `{PROJECT_ID}.{BQ_DATASET_ID}.{BQ_INPUT_TABLE}` AS

SELECT
  * EXCEPT (pseudo_random, income_bracket)
FROM (
  SELECT
    *,
    RAND() AS pseudo_random 
  FROM
    `bigquery-public-data.ml_datasets.census_adult_income` )
WHERE pseudo_random > {TRAIN_SPLIT}
"""
# Run the query
_ = bq_client.query(query)

## 为您的训练应用创建一个 Python 包

在执行批量预测任务之前，您需要对收入普查数据集进行随机森林分类模型的训练。您可以通过在 Vertex AI 中使用预构建容器来进行训练。为此，请按照以下步骤打包训练应用程序。

了解有关[为预构建容器创建 Python 训练应用程序](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)的更多信息。

### 准备源目录

创建一个名为 `python_package` 的源目录，并在其中创建一个名为 `trainer` 的子文件夹。接下来，在 `trainer` 文件夹中创建一个 `__init__.py` 文件，以将其变成一个包。

In [ ]:
!mkdir -p python_package
!mkdir -p python_package/trainer
!touch python_package/trainer/__init__.py

### 创建训练任务
在 `trainer/` 目录下创建一个名为 `task.py` 的模块，作为您的训练代码的入口点。

下面的训练器代码对训练集进行预处理，并将预处理的转换保存在 scikit-learn 流水线中。此外，对预处理后的训练数据进行 [随机森林模型训练](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)，并将其作为估计器添加到流水线中。在保存模型之后，将其上传到云存储桶以供部署使用。

使用 scikit-learn 流水线的优点是，它可以帮助您避免编写额外的预处理数据和生成预测的脚本。

了解更多关于 [scikit-learn 流水线](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)。

In [ ]:
%%writefile python_package/trainer/task.py
import os
import joblib
import argparse
from google.cloud import storage
from google.cloud import bigquery
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier

# Read environmental variables
PROJECT = os.getenv("CLOUD_ML_PROJECT_ID")
TRAINING_DATA_URI = os.getenv("AIP_TRAINING_DATA_URI")

# Set Bigquery Client
bq_client = bigquery.Client(project=PROJECT)
storage_client = storage.Client(project=PROJECT)

# Define the constants
TARGET = 'income_bracket'
ARTIFACTS_PATH = os.getenv("AIP_MODEL_DIR")
# Get the bucket name from the model dir
BUCKET_NAME = ARTIFACTS_PATH.replace("gs://","").split("/")[0]

MODEL_FILENAME = 'model.joblib' 
# Define the format of your input data, excluding the target column.
# These are the columns from the census data files.
COLUMNS = [
    'age',
    'workclass',
    'functional_weight',
    'education',
    'education_num',
    'marital_status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital_gain',
    'capital_loss',
    'hours_per_week',
    'native_country'
]
# Categorical columns are columns that need to be turned into a numerical value to be used by scikit-learn
CATEGORICAL_COLUMNS = [
    'workclass',
    'education',
    'marital_status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native_country'
]

# Function to fetch the data from BigQuery
def download_table(bq_table_uri: str):
    prefix = "bq://"
    if bq_table_uri.startswith(prefix):
        bq_table_uri = bq_table_uri[len(prefix):]

    table = bigquery.TableReference.from_string(bq_table_uri)
    rows = bq_client.list_rows(
        table,
    )
    return rows.to_dataframe(create_bqstorage_client=False)

# Function to upload local files to GCS
def upload_model(bucket_name: str,
                filename: str):
     # Upload the saved model file to GCS
    bucket = storage_client.get_bucket(bucket_name)
    storage_path = os.path.join(ARTIFACTS_PATH, filename)
    blob = storage.blob.Blob.from_string(storage_path, client=storage_client)
    blob.upload_from_filename(filename)
    

if __name__ == '__main__':
    # Load the training data
    X_train = download_table(TRAINING_DATA_URI)

    # Remove the column we are trying to predict ('income-level') from our features list
    # Convert the Dataframe to a lists of lists
    train_features = X_train.drop(TARGET, axis=1).to_numpy().tolist()
    # Create our training labels list, convert the Dataframe to a lists of lists
    train_labels = X_train[TARGET].to_numpy().tolist()

    # Since the census data set has categorical features, we need to convert
    # them to numerical values. We use a list of pipelines to convert each
    # categorical column and then use FeatureUnion to combine them before calling
    # the RandomForestClassifier.
    categorical_pipelines = []

    # Each categorical column needs to be extracted individually and converted to a numerical value.
    # To do this, each categorical column use a pipeline that extracts one feature column via
    # SelectKBest(k=1) and a LabelBinarizer() to convert the categorical value to a numerical one.
    # A scores array (created below) selects and extracts the feature column. The scores array is
    # created by iterating over the COLUMNS and checking if it is a CATEGORICAL_COLUMN.
    for i, col in enumerate(COLUMNS):
        if col in CATEGORICAL_COLUMNS:
            # Create a scores array to get the individual categorical column.
            # Example:
            #  data = [39, 'State-gov', 77516, 'Bachelors', 13, 'Never-married', 'Adm-clerical',
            #         'Not-in-family', 'White', 'Male', 2174, 0, 40, 'United-States']
            #  scores = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
            #
            # Returns: [['Sate-gov']]
            scores = []
            # Build the scores array
            for j in range(len(COLUMNS)):
                if i == j: # This column is the categorical column we want to extract.
                    scores.append(1) # Set to 1 to select this column
                else: # Every other column should be ignored.
                    scores.append(0)
            skb = SelectKBest(k=1)
            skb.scores_ = scores
            # Convert the categorical column to a numerical value
            lbn = LabelBinarizer()
            r = skb.transform(train_features)
            lbn.fit(r)
            # Create the pipeline to extract the categorical feature
            categorical_pipelines.append(
                ('categorical-{}'.format(i), Pipeline([
                    ('SKB-{}'.format(i), skb),
                    ('LBN-{}'.format(i), lbn)])))

    # Create pipeline to extract the numerical features
    skb = SelectKBest(k=6)
    # From COLUMNS use the features that are numerical
    skb.scores_ = [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0]
    categorical_pipelines.append(('numerical', skb))

    # Combine all the features using FeatureUnion
    preprocess = FeatureUnion(categorical_pipelines)

    # Create the classifier
    classifier = RandomForestClassifier()

    # Transform the features and fit them to the classifier
    classifier.fit(preprocess.transform(train_features), train_labels)

    # Create the overall model as a single pipeline
    pipeline = Pipeline([
        ('union', preprocess),
        ('classifier', classifier)
    ])

    # Save the pipeline locally
    joblib.dump(pipeline, MODEL_FILENAME)
    
    # Upload the locally saved model to GCS
    upload_model(bucket_name = BUCKET_NAME, 
                 filename=MODEL_FILENAME
                )

创建一个`setup.py`文件，告诉Setuptools如何创建源代码分发。您还可以在`setup.py`文件中指定应用程序的标准依赖项。Vertex AI使用pip在为您的作业分配的副本上安装您的训练应用程序。

了解更多关于[Setuptools](https://setuptools.readthedocs.io/en/latest/)。

In [ ]:
%%writefile python_package/setup.py

from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = ['pandas','pyarrow']

setup(
    name='trainer',
    version='0.1',
    packages=find_packages(),
    include_package_data=True,
    description='My training application.'
)

创建源代码分发

运行以下命令创建一个源代码分发，`dist/trainer-0.1.tar.gz`.

In [ ]:
!cd python_package && python3 setup.py sdist --formats=gztar

### 将源分发复制到云存储

要使用预构建的容器训练自定义分类模型，请将您的训练应用程序的源分发复制到云存储路径。在训练过程中，您可以通过`python_package_gcs_uri`参数让Vertex AI SDK定位包。

In [ ]:
!gsutil cp -r python_package/dist/* $BUCKET_URI/training_package/

## 创建并运行管道

您的管道已经做好了所有准备工作。在当前步骤中，您将创建一个 Vertex AI 管道，其中包括以下组件，每个组件都有自己的目的：

- `TabularDatasetCreateOp`: 在 Vertex AI 中创建一个新的托管表格数据集。
- `CustomPythonPackageTrainingJobRunOp`: 使用 Python 包在 Vertex AI 中创建和运行一个自定义训练作业。
- `ModelBatchPredictOp`: 在 Vertex AI 中创建一个批量预测作业，并等待其完成。

所有上述组件都是从`google-cloud-pipeline-components` Python 库导入的。了解更多关于[Google Cloud Pipeline Components](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.26/index.html)。

In [ ]:
# Define the pipeline


@pipeline(name="custom-model-bq-batch-prediction-pipeline")
def custom_model_bq_batch_prediction_pipeline(
    project: str,
    location: str,
    dataset_display_name: str,
    dataset_bq_source: str,
    training_job_dispaly_name: str,
    gcs_staging_directory: str,
    python_package_gcs_uri: str,
    python_package_module_name: str,
    training_split: float,
    test_split: float,
    training_container_uri: str,
    serving_container_uri: str,
    training_bigquery_destination: str,
    model_display_name: str,
    batch_prediction_display_name: str,
    batch_prediction_instances_format: str,
    batch_prediction_predictions_format: str,
    batch_prediction_source_uri: str,
    batch_prediction_destination_uri: str,
    batch_prediction_machine_type: str = "n1-standard-4",
    batch_prediction_batch_size: int = 1000,
):
    from google_cloud_pipeline_components.aiplatform import (
        CustomPythonPackageTrainingJobRunOp, ModelBatchPredictOp,
        TabularDatasetCreateOp)

    # Create the dataset
    dataset_create_op = TabularDatasetCreateOp(
        project=project,
        location=location,
        display_name=dataset_display_name,
        bq_source=dataset_bq_source,
    )

    # Run the training task
    train_op = CustomPythonPackageTrainingJobRunOp(
        display_name=training_job_dispaly_name,
        python_package_gcs_uri=python_package_gcs_uri,
        python_module_name=python_package_module_name,
        container_uri=training_container_uri,
        model_display_name=model_display_name,
        model_serving_container_image_uri=serving_container_uri,
        dataset=dataset_create_op.outputs["dataset"],
        base_output_dir=gcs_staging_directory,
        bigquery_destination=training_bigquery_destination,
        training_fraction_split=training_split,
        test_fraction_split=test_split,
        staging_bucket=gcs_staging_directory,
    )

    # Run the batch prediction task
    _ = ModelBatchPredictOp(
        project=project,
        location=location,
        model=train_op.outputs["model"],
        instances_format=batch_prediction_instances_format,
        bigquery_source_input_uri=batch_prediction_source_uri,
        predictions_format=batch_prediction_predictions_format,
        bigquery_destination_output_uri=batch_prediction_destination_uri,
        job_display_name=batch_prediction_display_name,
        machine_type=batch_prediction_machine_type,
        manual_batch_tuning_parameters_batch_size=batch_prediction_batch_size,
    )

### 编译管道

在定义完你的管道之后，将其编译成 JSON 或 YAML 格式的文件（`PIPELINE_FILE_NAME`）。

In [ ]:
# Compile the pipeline
compiler.Compiler().compile(
    pipeline_func=custom_model_bq_batch_prediction_pipeline,
    package_path=PIPELINE_FILE_NAME,
)

### 设置参数

现在，定义参数来运行您的管道。

要将所需参数传递给管道中的各个组件，您需要定义以下参数：
- `project`：Google Cloud 项目的项目 ID，其中需要运行管道。
- `location`：管道需要运行的区域。
- `dataset_display_name`：Vertex AI 中托管数据集资源的显示名称。
- `dataset_bq_source`：作为 Vertex AI 中托管数据集来源的 BigQuery 表 URI。
- `training_job_dispaly_name`：自定义 Python 包训练作业的显示名称。
- `gcs_staging_directory`：用于存储训练工件的 Vertex AI 暂存目录。
- `python_package_gcs_uri`：用于训练的 Python 包的 Cloud Storage 路径。
- `python_package_module_name`：Python 包内用于训练的模块名称（训练任务）。
- `training_split`：要用于训练的总数据的百分比。
- `test_split`：要用于测试的总数据的百分比。提供给 **CustomPythonPackageTrainingJobRunOp** 组件的拆分百分比参数应该始终总和为1。
- `training_container_uri`：用于训练模型的预构建容器映像 URI。
- `serving_container_uri`：用于在 Vertex AI 上为模型提供服务的预构建容器映像 URI。
- `training_bigquery_destination`：在训练期间要将训练数据写入的 BigQuery 项目位置。
- `model_display_name`：要部署在 Vertex AI 模型注册表中的模型的显示名称。
- `batch_prediction_display_name`：批量预测作业的显示名称。
- `batch_prediction_instances_format`：用于批量预测的输入实例格式。
- `batch_prediction_predictions_format`：来自批量预测结果的格式。
- `batch_prediction_source_uri`：输入数据的来源 URI。
- `batch_prediction_destination_uri`：批量预测结果需要存储的目标 URI。

**注意：**虽然提供了测试拆分百分比，但测试数据在训练过程中不会被使用。这些测试数据与之前为批量预测创建的测试数据不同。

In [ ]:
# Define the parameters for running the pipeline
parameters = {
    "project": PROJECT_ID,
    "location": REGION,
    "dataset_display_name": DATASET_DISPLAY_NAME,
    "dataset_bq_source": DATA_SOURCE,
    "training_job_dispaly_name": TRAINING_JOB_DISPLAY_NAME,
    "gcs_staging_directory": BUCKET_URI,
    "python_package_gcs_uri": f"{BUCKET_URI}/training_package/trainer-0.1.tar.gz",
    "python_package_module_name": "trainer.task",
    "training_split": TRAIN_SPLIT,
    "test_split": 1 - TRAIN_SPLIT,
    "training_container_uri": TRAINING_CONTAINER,
    "serving_container_uri": SERVING_CONTAINER,
    "training_bigquery_destination": f"bq://{PROJECT_ID}",
    "model_display_name": MODEL_DISPLAY_NAME,
    "batch_prediction_display_name": BATCH_PREDICTION_JOB_NAME,
    "batch_prediction_instances_format": "bigquery",
    "batch_prediction_predictions_format": "bigquery",
    "batch_prediction_source_uri": f"bq://{PROJECT_ID}.{BQ_DATASET_ID}.{BQ_INPUT_TABLE}",
    "batch_prediction_destination_uri": f"bq://{PROJECT_ID}.{BQ_DATASET_ID}",
}

### 运行管道

创建一个Vertex AI Pipeline作业，并使用`PipelineJob`类来运行它。

`PipelineJob`类需要以下参数：

- `display_name`：Vertex AI Pipeline的显示名称。
- `template_path`：PipelineJob或PipelineSpec JSON（或YAML）文件的路径。
- `parameter_values`：控制管道运行的运行时参数名称与其值的映射。
- `enable_caching`：是否开启运行缓存。

从[Vertex AI PipelineJob文档](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.PipelineJob)中了解更多关于`PipelineJob`类的信息。

In [ ]:
# Create a Vertex AI Pipeline job
job = aiplatform.PipelineJob(
    display_name=PIPELINE_DISPLAY_NAME,
    template_path=PIPELINE_FILE_NAME,
    parameter_values=parameters,
    enable_caching=True,
)
# Run the pipeline job
job.run(service_account=SERVICE_ACCOUNT)

从预测表中获取结果

在Vertex AI管道作业成功完成后，从批量预测中获取结果并存入 dataframe。

### 获取表名

运行下面的单元格以从管道作业的工件详情中获取预测表的名称。

In [ ]:
OUTPUT_TABLE = None
# Load the batch prediction job details using the display name
[batch_prediction_job] = aiplatform.BatchPredictionJob.list(
    filter=f'display_name="{BATCH_PREDICTION_JOB_NAME}"'
)
# Fetch the name of the output table
OUTPUT_TABLE = batch_prediction_job.output_info.bigquery_output_table
print("Predictions table ID:", OUTPUT_TABLE)

### 查询结果表

使用以下单元格从预测表中检索指定数量的行。

In [ ]:
# Specify the needed no.of rows
ROWS = 10
# Define the query
query = f"""
    Select prediction from `{PROJECT_ID}.{BQ_DATASET_ID}.{OUTPUT_TABLE}` limit {ROWS}
"""
# Fetch the data into a dataframe
df = bq_client.query(query).to_dataframe()
# Display the dataframe
df

获取用于删除的资源

使用各个资源的显示名称，加载在清理步骤中创建的资源。

In [ ]:
# Load the Vertex AI tabular dataset using the display name
[dataset] = aiplatform.TabularDataset.list(
    filter=f'display_name="{DATASET_DISPLAY_NAME}"'
)

# Load the Vertex AI model resource using the display name
[model] = aiplatform.Model.list(filter=f'display_name="{MODEL_DISPLAY_NAME}"')

# Load the custom training job using the display name
[training_job] = aiplatform.CustomPythonPackageTrainingJob.list(
    filter=f'display_name="{TRAINING_JOB_DISPLAY_NAME}"'
)

清理

要清理此项目中使用的所有谷歌云资源，您可以[删除用于本教程的谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- Vertex AI Pipeline Job
- Vertex AI TabularDataset
- Vertex AI Model
- Vertex AI Training job
- Vertex AI Batch Prediction job
- BigQuery dataset
- Cloud Storage bucket（将`delete_bucket`设置为**True**以删除Cloud Storage桶）

In [ ]:
delete_bucket = False

# Delete the Vertex AI Pipeline job
job.delete()

# Delete the Vertex AI TabularDataset
dataset.delete()

# Delete the Vertex AI Model
model.delete()

# Delete the Vertex AI Training job
training_job.delete()

# Delete the Vertex AI Batch prediction job
batch_prediction_job.delete()

# Delete the BigQuery dataset
! bq rm -r -f -d $PROJECT_ID:$BQ_DATASET_ID

# Delete Cloud Storage objects
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI